In [1]:
######### 01 본문 pdf word 변환 시작 ########################################################################
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.shared import OxmlElement, qn


# 오늘 날짜 문자열 생성 (YYYY-MM-DD 형식)
today_str = datetime.now().strftime('%Y%m%d')

# 파일 경로 설정
EXCEL_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list_{today_str}.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"

today_dir = os.path.join(OUTPUT_DIR, today_str)


# PDF와 Word 파일용 폴더 생성
pdf_dir = os.path.join(today_dir, "pdf")
doc_dir = os.path.join(today_dir, "doc")

# 폴더 생성
for dir_path in [today_dir, pdf_dir, doc_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"폴더 생성 완료: {dir_path}")

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# macOS 폰트 설정
print("macOS 폰트 검색 중...")
font_registered = False

# macOS에서 사용 가능한 한글 폰트들
possible_fonts = [
    '/System/Library/Fonts/AppleSDGothicNeo.ttc',
    '/System/Library/Fonts/AppleGothic.ttf',
    '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
    '/Library/Fonts/AppleGothic.ttf',
    '/Library/Fonts/NanumGothic.ttf',
    '/System/Library/Fonts/STHeiti Light.ttc',
    '/System/Library/Fonts/STHeiti Medium.ttc'
]

for font_path in possible_fonts:
    if os.path.exists(font_path):
        try:
            print(f"폰트 발견: {font_path}")
            if font_path.endswith('.ttc'):
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
            else:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
            font_registered = True
            print(f"폰트 등록 성공: {font_path}")
            break
        except Exception as e:
            print(f"폰트 등록 실패({font_path}): {e}")
            continue

# 폰트 등록에 실패한 경우 기본 폰트 사용
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 기본 폰트를 사용합니다.")
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# PDF 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 공통 함수들
def add_hyperlink(paragraph, url, text, bold=True, font_size=14):
    """Word 문서에 하이퍼링크 추가"""
    from docx.oxml.shared import OxmlElement, qn
    
    # 하이퍼링크 관계 생성
    part = paragraph.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)
    
    # 하이퍼링크 XML 요소 생성
    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('r:id'), r_id)
    
    # 텍스트 실행 요소 생성
    new_run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')
    
    # 굵은 폰트 설정
    if bold:
        bold_elem = OxmlElement('w:b')
        rPr.append(bold_elem)
    
    # 폰트 크기 설정
    sz = OxmlElement('w:sz')
    sz.set(qn('w:val'), str(font_size * 2))  # Word에서는 폰트 크기를 2배로 설정
    rPr.append(sz)
    
    # 링크 스타일 설정 (파란색, 밑줄)
    color = OxmlElement('w:color')
    color.set(qn('w:val'), '0000FF')
    rPr.append(color)
    
    u = OxmlElement('w:u')
    u.set(qn('w:val'), 'single')
    rPr.append(u)
    
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)
    
    # 문단에 하이퍼링크 추가
    paragraph._p.append(hyperlink)
    
    return hyperlink
def clean_text(text):
    """HTML 태그 제거 및 텍스트 정리"""
    text = str(text)
    text = text.replace('<h3>', '').replace('</h3>', '').replace('#', '').replace('**', '')
    
    # <a href= 또는 </a>를 포함하는 문장 전체를 삭제
    import re
    
    # 문장 단위로 분리
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # 링크 태그가 없는 문장만 유지
    cleaned_sentences = [s for s in sentences if '<a href=' not in s and '</a>' not in s]
    
    # 다시 문장들을 합치기
    text = ' '.join(cleaned_sentences)
    
    return text

def create_safe_filename(title):
    """파일명에 사용할 수 없는 문자 제거"""
    return re.sub(r'[\\/*?:"<>|]', '', title)

def extract_keywords_from_title(title):
    """제목에서 키워드 추출"""
    keywords = []
    
    # 제목을 공백, 쉼표, 하이픈 등으로 분리
    # 다양한 구분자를 고려하여 단어 분리
    import re
    words = re.split(r'[\s,.\-_\(\)\[\]]+', title)
    
    # 불필요한 단어들 제거
    exclude_words = ['pdf', 'docx', 'doc']
    
    for word in words:
        word = word.strip()
        # 2글자 이상이고, 불필요한 단어가 아니며, 한글 또는 영문으로만 구성된 경우
        if (len(word) >= 2 and 
            word not in exclude_words and 
            re.match(r'^[가-힣a-zA-Z]+$', word)):
            keywords.append(word)
    
    # 중복 제거
    unique_keywords = []
    for keyword in keywords:
        if keyword not in unique_keywords:
            unique_keywords.append(keyword)
    
    return ', '.join(unique_keywords)

def process_body_text(body):
    """본문 텍스트 처리 - 구분선 추가 및 헤딩 처리"""
    body_lines = body.split('\n')
    processed_lines = []
    
    for line in body_lines:
        if '<h2>' in line:
            processed_lines.append("---------------------------------------------------------------------")
        processed_lines.append(line)
    
    return '\n'.join(processed_lines)

def add_line_to_word_doc(doc, text, style_type='normal'):
    """Word 문서에 텍스트 추가"""
    if style_type == 'separator':
        # 구분선 추가
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif style_type == 'heading':
        # 제목 스타일 추가
        p = doc.add_paragraph()
        run = p.add_run(text)
        run.bold = True
        run.underline = True
        run.font.size = Pt(14)
        p.space_before = Pt(16)
        p.space_after = Pt(8)
    elif style_type == 'title':
        # 메인 제목
        p = doc.add_heading(text, level=1)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    else:
        # 일반 문단
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        p.paragraph_format.first_line_indent = Inches(0.28)
        p.paragraph_format.space_before = Pt(12)
        p.paragraph_format.space_after = Pt(12)

# 각 행에 대해 PDF와 Word 파일 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = clean_text(row['body'])
        
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 안전한 파일명 생성
        safe_title = create_safe_filename(title)
        
        # PDF 파일 경로
        pdf_path = os.path.join(pdf_dir, f"{safe_title}_대표 기도문 모음 나눔터.pdf")
        # Word 파일 경로
        word_path = os.path.join(doc_dir, f"{safe_title}_대표 기도문 모음 나눔터.docx")
        
        # 제목에서 키워드 추출
        keywords = extract_keywords_from_title(title)
        print(f"추출된 키워드: {keywords}")
        
        # === PDF 생성 ===
        # PDF 메타데이터를 위한 정보 준비
        from reportlab.lib.units import inch
        from reportlab.platypus import SimpleDocTemplate
        
        doc_pdf = SimpleDocTemplate(
            pdf_path,
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72,
            title=title,
            subject="대표 기도문 모음",
            author="대표 기도문 나눔터",
            creator="기도문 모음",
            keywords=keywords
        )
        
        story = []
        
        # PDF 제목 추가
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # === Word 문서 생성 ===
        doc_word = Document()
        
        # Word 문서 제목 추가
        add_line_to_word_doc(doc_word, title, 'title')
        doc_word.add_paragraph()  # 빈 줄 추가
        
        # 본문 처리
        processed_body = process_body_text(body)
        
        # <h2> 태그 처리를 위한 정규 표현식
        h2_pattern = re.compile(r'<h2>(.*?)</h2>')
        
        # 구분선 스타일 정의 (PDF용)
        separator_style = ParagraphStyle(
            name='Separator',
            parent=styles['Normal'],
            fontName=font_name,
            alignment=TA_JUSTIFY
        )
        
        # 본문을 특별 태그로 변환
        styled_body = ""
        for para in processed_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                styled_body += para + "\n"
                continue
                
            matches = h2_pattern.findall(para)
            if matches:
                styled_para = para
                for match in matches:
                    original = f"<h2>{match}</h2>"
                    styled_para = styled_para.replace(original, f"##HEADING_START##{match}##HEADING_END##")
                styled_body += styled_para + "\n"
            else:
                styled_body += para + "\n"
        
        # 문단 분석 및 처리
        paragraphs = []
        for para in styled_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                paragraphs.append((para, "separator"))
                continue
                
            if "##HEADING_START##" in para:
                parts = para.split("##HEADING_START##")
                
                if parts[0]:
                    paragraphs.append((parts[0], "normal"))
                
                for i in range(1, len(parts)):
                    if "##HEADING_END##" in parts[i]:
                        heading_parts = parts[i].split("##HEADING_END##")
                        if heading_parts[0]:
                            paragraphs.append((heading_parts[0], "heading"))
                        if len(heading_parts) > 1 and heading_parts[1]:
                            paragraphs.append((heading_parts[1], "normal"))
            else:
                paragraphs.append((para, "normal"))
        
        # PDF와 Word 문서에 내용 추가
        for para_text, para_type in paragraphs:
            if para_text.strip() == '':
                story.append(Spacer(1, 0.1 * inch))
                continue
            
            # PDF에 추가
            if para_type == "separator":
                story.append(Paragraph(para_text, separator_style))
                add_line_to_word_doc(doc_word, para_text, 'separator')
            else:
                para_text_clean = para_text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                para_text_clean = re.sub(r' +', ' ', para_text_clean)
                
                if para_type == "heading":
                    heading_style = ParagraphStyle(
                        name='Heading2',
                        parent=styles['Normal_Justified'],
                        fontName=font_name,
                        fontSize=14,
                        fontWeight='bold',
                        underline=True,
                        spaceBefore=16,
                        spaceAfter=8
                    )
                    story.append(Paragraph(para_text_clean, heading_style))
                    add_line_to_word_doc(doc_word, para_text, 'heading')
                else:
                    story.append(Paragraph(para_text_clean, styles['Normal_Justified']))
                    add_line_to_word_doc(doc_word, para_text, 'normal')
        
        # PDF와 Word 문서 마지막에 링크 추가
        # PDF에 링크 추가
        story.append(Spacer(1, 0.3 * inch))  # 여백 추가
        link_style = ParagraphStyle(
            name='LinkStyle',
            parent=styles['Normal'],
            fontName=font_name,
            fontSize=14,  # 폰트 크기 증가
            spaceBefore=20,
            spaceAfter=12,
            alignment=1,  # 가운데 정렬 (TA_CENTER)
            leading=20    # 줄간격 넓히기
        )
        
        # PDF 링크 텍스트 (굵은 폰트, 클릭 가능한 링크)
        link_text = '<b>더 많은 기도문 보러가기:<br/>[대표 기도문 나눔터] <link href="https://prayer-church.co.kr/" color="blue">https://prayer-church.co.kr/</link></b>'
        story.append(Paragraph(link_text, link_style))
        
        # Word 문서에 링크 추가
        doc_word.add_paragraph()  # 빈 줄 추가
        link_paragraph = doc_word.add_paragraph()
        
        # 줄간격 설정 (Word)
        from docx.shared import Pt
        link_paragraph.paragraph_format.line_spacing = Pt(20)
        
        # 첫 번째 줄 - "더 많은 기도문 보러가기:" (굵은 폰트)
        run1 = link_paragraph.add_run("더 많은 기도문 보러가기:")
        run1.bold = True
        run1.font.size = Pt(14)
        
        # 줄바꿈 추가
        link_paragraph.add_run("\n")
        
        # 두 번째 줄 - "[대표 기도문 나눔터]" (굵은 폰트)
        run2 = link_paragraph.add_run("[대표 기도문 나눔터] ")
        run2.bold = True
        run2.font.size = Pt(14)
        
        # Word에 하이퍼링크 추가 (굵은 폰트)
        try:
            # 기존 텍스트 뒤에 하이퍼링크 추가
            hyperlink_run = link_paragraph.add_run()
            hyperlink_run.bold = True
            hyperlink_run.font.size = Pt(14)
            hyperlink = add_hyperlink(link_paragraph, "https://prayer-church.co.kr/", "https://prayer-church.co.kr/")
        except:
            # 하이퍼링크 추가 실패시 일반 텍스트로 추가 (굵은 폰트)
            run3 = link_paragraph.add_run("https://prayer-church.co.kr/")
            run3.bold = True
            run3.font.size = Pt(14)
        
        link_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # PDF 파일 저장
        try:
            doc_pdf.build(story)
            print(f"PDF 생성 완료: {pdf_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
        
        # Word 문서 속성 설정 (SEO 메타데이터)
        core_props = doc_word.core_properties
        core_props.title = title
        core_props.subject = "기도문 모음"
        core_props.author = "대표 기도문 나눔터"
        core_props.keywords = keywords
        core_props.comments = f"{title} - 더 많은 기도문은 https://prayer-church.co.kr/ 에서 확인하세요"
        core_props.category = "종교/신앙"
        core_props.content_status = "최종"
        
        # Word 추가 사용자 정의 속성 (대안 방법)
        try:
            # Document의 part를 통해 custom properties 접근
            from docx.opc.part import PartFactory
            from docx.opc.constants import CONTENT_TYPE
            
            # 사용자 정의 속성을 comments에 추가로 포함
            extended_comments = f"{core_props.comments}\n웹사이트: https://prayer-church.co.kr/\n콘텐츠 유형: 기도문\nSEO 설명: {title[:100]}... - 대표 기도문 나눔터에서 제공하는 기도문 모음"
            core_props.comments = extended_comments
            print(f"메타데이터 설정 완료")
        except Exception as e:
            print(f"확장 메타데이터 설정 실패: {e}")
        
        # Word 파일 저장
        try:
            doc_word.save(word_path)
            print(f"Word 파일 생성 완료: {word_path}")
        except Exception as e:
            print(f"Word 파일 생성 실패 ({safe_title}): {e}")
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print(f"\n모든 변환 작업이 완료되었습니다.")
print(f"PDF 파일 저장 위치: {pdf_dir}")
print(f"Word 파일 저장 위치: {doc_dir}")
print(f"------ 01. 모든 파일 변환 완료 -------")

######### 01 본문 pdf word 변환 종료 ########################################################################

######### 02 pdf word 파일 업로드 시작 ########################################################################

import os
import time
import requests
import pandas as pd
from pathlib import Path
import mimetypes
import re
from urllib.parse import urlparse
import base64
from PyPDF2 import PdfReader
from docx import Document
import json

# WordPress 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"
date = time.strftime("%Y%m%d")

# 인증 헤더 생성
credentials = base64.b64encode(f"{USERNAME}:{APP_PASSWORD}".encode()).decode()
headers = {
    'Authorization': f'Basic {credentials}',
    'Accept': 'application/json'
}

def extract_pdf_metadata(file_path):
    """PDF 파일에서 메타데이터 추출"""
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            metadata = pdf_reader.metadata
            
            if metadata:
                return {
                    'title': metadata.get('/Title', ''),
                    'subject': metadata.get('/Subject', ''),
                    'author': metadata.get('/Author', ''),
                    'creator': metadata.get('/Creator', ''),
                    'keywords': metadata.get('/Keywords', ''),
                    'producer': metadata.get('/Producer', '')
                }
    except Exception as e:
        print(f"PDF 메타데이터 추출 실패 ({file_path}): {e}")
    return {}

def extract_docx_metadata(file_path):
    """Word 문서에서 메타데이터 추출"""
    try:
        doc = Document(file_path)
        core_props = doc.core_properties
        
        return {
            'title': core_props.title or '',
            'subject': core_props.subject or '',
            'author': core_props.author or '',
            'keywords': core_props.keywords or '',
            'comments': core_props.comments or '',
            'category': core_props.category or '',
            'content_status': core_props.content_status or ''
        }
    except Exception as e:
        print(f"Word 메타데이터 추출 실패 ({file_path}): {e}")
    return {}

def create_seo_metadata_from_file(file_path, filename):
    """파일에서 추출한 메타데이터로 SEO 정보 생성"""
    file_ext = os.path.splitext(filename)[1].lower()
    
    # 파일별 메타데이터 추출
    if file_ext == '.pdf':
        metadata = extract_pdf_metadata(file_path)
    elif file_ext in ['.doc', '.docx']:
        metadata = extract_docx_metadata(file_path)
    else:
        metadata = {}
    
    # 메타데이터에서 정보 추출
    if metadata and metadata.get('title'):
        title = metadata['title']
        
        # 키워드에서 SEO 태그 추출
        keywords = metadata.get('keywords', '')
        if keywords:
            # 키워드를 분리하여 SEO 친화적으로 처리
            keyword_list = [k.strip() for k in keywords.split(',') if k.strip()]
            alt_text = f"{title} - {', '.join(keyword_list[:3])}"
        else:
            alt_text = f"{title} - 대표 기도문 나눔터"
        
        # 주제나 설명에서 캡션 생성
        subject = metadata.get('subject', '')
        comments = metadata.get('comments', '')
        
        if comments and 'https://prayer-church.co.kr/' in comments:
            # 기존 설명이 있으면 활용
            caption = f"{title} - 대표 기도문 나눔터에서 제공하는 기도문 모음입니다."
        else:
            caption = f"{title} - {subject}" if subject else f"{title} - 대표 기도문 나눔터"
        
        return title, alt_text, caption, metadata
    
    # 메타데이터가 없으면 파일명 기반으로 생성
    return create_fallback_seo_metadata(filename)

def create_fallback_seo_metadata(filename):
    """파일명 기반 SEO 메타데이터 생성 (메타데이터가 없을 때)"""
    name, ext = os.path.splitext(filename)
    
    # "_대표 기도문 모음 나눔터" 제거하여 제목 추출
    title = name.replace('_대표 기도문 모음 나눔터', '').strip()
    
    # SEO 친화적 제목 생성
    title = title.replace('-', ' ').replace('_', ' ').strip()
    
    # 대체 텍스트 생성
    alt_text = f"{title} - 대표 기도문 모음 나눔터"
    
    # 캡션 생성
    if ext.lower() == '.pdf':
        caption = f"{title} - 대표 기도문 모음 나눔터에서 제공하는 PDF 기도문입니다."
    elif ext.lower() in ['.doc', '.docx']:
        caption = f"{title} - 대표 기도문 모음 나눔터에서 제공하는 Word 기도문입니다."
    else:
        caption = f"{title} - 대표 기도문 모음 나눔터"
    
    return title, alt_text, caption, {}

def clean_filename_for_seo(filename):
    """파일명을 SEO 친화적으로 변환"""
    # 확장자 분리
    name, ext = os.path.splitext(filename)
    
    # "_대표 기도문 모음 나눔터" 부분 제거
    name = name.replace('_대표 기도문 모음 나눔터', '')
    
    # 특수문자 제거 및 하이픈으로 변환
    name = re.sub(r'[^a-zA-Z0-9가-힣\s]', '-', name)
    name = re.sub(r'\s+', '-', name)  # 공백을 하이픈으로
    name = re.sub(r'-+', '-', name)  # 연속된 하이픈 제거
    name = name.strip('-').lower()
    
    return f"{name}_대표_기도문_나눔터{ext}"

def upload_file_to_wordpress(file_path):
    """워드프레스에 파일 업로드"""
    try:
        # 파일 정보 확인
        if not os.path.exists(file_path):
            return None, f"파일이 존재하지 않습니다: {file_path}"
        
        filename = os.path.basename(file_path)
        seo_filename = clean_filename_for_seo(filename)
        
        # 파일 메타데이터에서 SEO 정보 추출
        title, alt_text, caption, file_metadata = create_seo_metadata_from_file(file_path, filename)
        
        # MIME 타입 확인
        mime_type, _ = mimetypes.guess_type(file_path)
        if not mime_type:
            mime_type = 'application/octet-stream'
        
        # 파일 읽기
        with open(file_path, 'rb') as file:
            files = {
                'file': (seo_filename, file, mime_type)
            }
            
            # 업로드 요청
            upload_headers = headers.copy()
            upload_headers.pop('Accept', None)  # multipart 요청에서는 제거
            
            response = requests.post(
                f"{WP_API}/media",
                headers=upload_headers,
                files=files
            )
        
        if response.status_code == 201:
            media_data = response.json()
            media_id = media_data['id']
            
            # 추가 SEO 메타데이터 생성
            description = caption
            if file_metadata.get('keywords'):
                # 키워드를 설명에 추가
                keywords = file_metadata['keywords']
                description += f" 관련 키워드: {keywords}"
            
            # WordPress 미디어 메타데이터 업데이트
            update_data = {
                'title': title,
                'alt_text': alt_text,
                'caption': caption,
                'description': description
            }
            
            # 사용자 정의 필드 추가 (가능한 경우)
            if file_metadata:
                # 메타데이터를 meta 필드에 추가
                meta_fields = {}
                if file_metadata.get('author'):
                    meta_fields['file_author'] = file_metadata['author']
                if file_metadata.get('keywords'):
                    meta_fields['file_keywords'] = file_metadata['keywords']
                if file_metadata.get('subject'):
                    meta_fields['file_subject'] = file_metadata['subject']
                
                if meta_fields:
                    update_data['meta'] = meta_fields
            
            update_response = requests.post(
                f"{WP_API}/media/{media_id}",
                headers=headers,
                json=update_data
            )
            
            if update_response.status_code == 200:
                updated_data = update_response.json()
                return {
                    'id': media_id,
                    'url': media_data['source_url'],
                    'title': title,
                    'alt_text': alt_text,
                    'caption': caption,
                    'description': description,
                    'file_metadata': file_metadata,
                    'status': '성공'
                }, None
            else:
                return {
                    'id': media_id,
                    'url': media_data['source_url'],
                    'title': title,
                    'alt_text': alt_text,
                    'caption': caption,
                    'description': description,
                    'file_metadata': file_metadata,
                    'status': '업로드 성공, 메타데이터 업데이트 실패'
                }, None
        else:
            return None, f"업로드 실패: {response.status_code} - {response.text}"
            
    except Exception as e:
        return None, f"오류 발생: {str(e)}"

def process_folder(folder_path):
    """폴더 내 모든 파일 처리"""
    results = []
    
    if not os.path.exists(folder_path):
        print(f"폴더가 존재하지 않습니다: {folder_path}")
        return results
    
    # 지원되는 파일 확장자
    supported_extensions = ['.pdf', '.doc', '.docx', '.jpg', '.jpeg', '.png', '.gif', '.webp']
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            _, ext = os.path.splitext(file)
            
            if ext.lower() in supported_extensions:
                print(f"업로드 중: {file}")
                result, error = upload_file_to_wordpress(file_path)
                
                if result:
                    # 결과에 파일 메타데이터 정보 추가
                    file_metadata = result.get('file_metadata', {})
                    metadata_summary = {
                        '파일_제목': file_metadata.get('title', ''),
                        '파일_작성자': file_metadata.get('author', ''),
                        '파일_키워드': file_metadata.get('keywords', ''),
                        '파일_설명': file_metadata.get('comments', '')[:100] + '...' if file_metadata.get('comments') else ''
                    }
                    
                    results.append({
                        '파일명': file,
                        '파일경로': file_path,
                        '상태': result['status'],
                        'ID': result['id'],
                        'URL': result['url'],
                        '제목': result['title'],
                        '대체텍스트': result['alt_text'],
                        '캡션': result['caption'],
                        '설명': result.get('description', ''),
                        '원본_파일_제목': metadata_summary['파일_제목'],
                        '원본_파일_작성자': metadata_summary['파일_작성자'],
                        '원본_파일_키워드': metadata_summary['파일_키워드'],
                        '원본_파일_설명': metadata_summary['파일_설명']
                    })
                    print(f"✓ 성공: {file}")
                else:
                    results.append({
                        '파일명': file,
                        '파일경로': file_path,
                        '상태': f'실패 - {error}',
                        'ID': '',
                        'URL': '',
                        '제목': '',
                        '대체텍스트': '',
                        '캡션': '',
                        '설명': '',
                        '원본_파일_제목': '',
                        '원본_파일_작성자': '',
                        '원본_파일_키워드': '',
                        '원본_파일_설명': ''
                    })
                    print(f"✗ 실패: {file} - {error}")
                
                # 너무 빠른 요청 방지
                time.sleep(1)
            else:
                print(f"지원하지 않는 파일 형식: {file}")
    
    return results

def main():
    """메인 실행 함수"""
    base_path = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{date}"
    doc_folder = os.path.join(base_path, "doc")
    pdf_folder = os.path.join(base_path, "pdf")
    report_folder = os.path.join(base_path, "report")
    
    # 결과 수집
    all_results = []
    
    print("=== WordPress 미디어 업로드 시작 ===")
    print(f"날짜: {date}")
    print("생성된 파일의 메타데이터를 활용하여 SEO 최적화합니다.")
    
    # DOC 폴더 처리
    print(f"\n1. DOC 폴더 처리: {doc_folder}")
    doc_results = process_folder(doc_folder)
    all_results.extend(doc_results)
    
    # PDF 폴더 처리  
    print(f"\n2. PDF 폴더 처리: {pdf_folder}")
    pdf_results = process_folder(pdf_folder)
    all_results.extend(pdf_results)
    
    # 리포트 폴더 생성
    os.makedirs(report_folder, exist_ok=True)
    
    # 엑셀 리포트 생성
    if all_results:
        df = pd.DataFrame(all_results)
        excel_path = os.path.join(report_folder, f"media_upload_report_{date}.xlsx")
        
        # 엑셀 스타일링
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='업로드 리포트')
            
            # 워크시트 가져오기
            worksheet = writer.sheets['업로드 리포트']
            
            # 열 너비 자동 조정
            for column in worksheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = min(max_length + 2, 60)
                worksheet.column_dimensions[column_letter].width = adjusted_width
        
        print(f"총 처리 파일: {len(all_results)}개")
        print(f"성공: {len([r for r in all_results if '성공' in r['상태']])}개")
        print(f"실패: {len([r for r in all_results if '실패' in r['상태']])}개")
        print(f"리포트 저장 위치: {excel_path}")
        print(f"========= 02.모든 파일 업로드 완료 =========")
       
        # 메타데이터 활용 통계
        metadata_count = len([r for r in all_results if r['원본_파일_제목']])
        print(f"메타데이터 활용 파일: {metadata_count}개")
    else:
        print("\n처리할 파일이 없습니다.")

if __name__ == "__main__":
    main()

######### 02 pdf word 파일 업로드 종료 ########################################################################

폴더 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf
폴더 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc
macOS 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list_20250723.xlsx
총 80 개의 행이 로드되었습니다.

처리 중인 행 1/80 - 제목: 새벽예배 대표기도문 10월...
추출된 키워드: 새벽예배, 대표기도문
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/새벽예배 대표기도문 10월_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/새벽예배 대표기도문 10월_대표 기도문 모음 나눔터.docx

처리 중인 행 2/80 - 제목: 새벽예배 대표기도문 11월...
추출된 키워드: 새벽예배, 대표기도문
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf

PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/새벽예배 대표기도문 9월 마지막주_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/새벽예배 대표기도문 9월 마지막주_대표 기도문 모음 나눔터.docx

처리 중인 행 28/80 - 제목: 새벽예배 대표기도문 10월 첫째주...
추출된 키워드: 새벽예배, 대표기도문, 첫째주
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/새벽예배 대표기도문 10월 첫째주_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/새벽예배 대표기도문 10월 첫째주_대표 기도문 모음 나눔터.docx

처리 중인 행 29/80 - 제목: 새벽예배 대표기도문 10월 둘째주...
추출된 키워드: 새벽예배, 대표기도문, 둘째주
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/새벽예배 대표기도문 10월 둘째주_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/새벽예배 대표기도문 10월 둘째주_대표 기도문 모음 나눔터.docx

처리 중인 행 30/80 - 제목: 새벽예배 대표기도문 10월 셋째주...
추출된 키워드: 새벽예배, 대표기도문, 셋째주
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/새벽예배 대표기도

PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/9월 첫째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/9월 첫째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx

처리 중인 행 56/80 - 제목: 9월 둘째주 주일 낮예배 대표 기도문...
추출된 키워드: 둘째주, 주일, 낮예배, 대표, 기도문
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/9월 둘째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/9월 둘째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx

처리 중인 행 57/80 - 제목: 9월 셋째주 주일 낮예배 대표 기도문...
추출된 키워드: 셋째주, 주일, 낮예배, 대표, 기도문
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/pdf/9월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
메타데이터 설정 완료
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc/9월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx

처리 중인 행 58/80 - 제목: 9월 넷째주 주일 낮예배 대표 기도문...
추출된 키워드: 넷째주, 주일, 낮예배, 대표, 기도문
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsens

=== WordPress 미디어 업로드 시작 ===
날짜: 20250723
생성된 파일의 메타데이터를 활용하여 SEO 최적화합니다.

1. DOC 폴더 처리: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/doc
업로드 중: 새벽예배 대표기도문 9월 셋째주_대표 기도문 모음 나눔터.docx
✓ 성공: 새벽예배 대표기도문 9월 셋째주_대표 기도문 모음 나눔터.docx
업로드 중: 새벽예배 대표기도문 12월 셋째주_대표 기도문 모음 나눔터.docx
✓ 성공: 새벽예배 대표기도문 12월 셋째주_대표 기도문 모음 나눔터.docx
업로드 중: 새벽예배 대표기도문 10월 마지막주_대표 기도문 모음 나눔터.docx
✓ 성공: 새벽예배 대표기도문 10월 마지막주_대표 기도문 모음 나눔터.docx
업로드 중: 7월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
✓ 성공: 7월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
업로드 중: 8월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
✓ 성공: 8월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
업로드 중: 새벽예배 대표기도문 12월_대표 기도문 모음 나눔터.docx
✓ 성공: 새벽예배 대표기도문 12월_대표 기도문 모음 나눔터.docx
업로드 중: 새벽예배 대표기도문 6월 마지막주_대표 기도문 모음 나눔터.docx
✓ 성공: 새벽예배 대표기도문 6월 마지막주_대표 기도문 모음 나눔터.docx
업로드 중: 6월 3째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
✓ 성공: 6월 3째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx
업로드 중: 주일 낮예배 대표 기도문 6월_대표 기도문 모음 나눔터.docx
✓ 성공: 주일 낮예배 대표 기도문 6월_대표 기도문 모음 나눔터.docx
업로드 중: 11월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.docx


업로드 중: 새벽예배 대표기도문 12월 둘째주_대표 기도문 모음 나눔터.pdf
✓ 성공: 새벽예배 대표기도문 12월 둘째주_대표 기도문 모음 나눔터.pdf
업로드 중: 12월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
✓ 성공: 12월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
업로드 중: 새벽예배 대표기도문 12월 넷째주_대표 기도문 모음 나눔터.pdf
✓ 성공: 새벽예배 대표기도문 12월 넷째주_대표 기도문 모음 나눔터.pdf
업로드 중: 6월 2째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
✓ 성공: 6월 2째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
업로드 중: 8월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
✓ 성공: 8월 넷째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
업로드 중: 새벽예배 대표기도문 11월_대표 기도문 모음 나눔터.pdf
✓ 성공: 새벽예배 대표기도문 11월_대표 기도문 모음 나눔터.pdf
업로드 중: 주일 낮예배 대표 기도문 2월_대표 기도문 모음 나눔터.pdf
✓ 성공: 주일 낮예배 대표 기도문 2월_대표 기도문 모음 나눔터.pdf
업로드 중: 주일 낮예배 대표 기도문 4월_대표 기도문 모음 나눔터.pdf
✓ 성공: 주일 낮예배 대표 기도문 4월_대표 기도문 모음 나눔터.pdf
업로드 중: 7월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
✓ 성공: 7월 셋째주 주일 낮예배 대표 기도문_대표 기도문 모음 나눔터.pdf
업로드 중: 새벽예배 대표기도문 11월 마지막주_대표 기도문 모음 나눔터.pdf
✓ 성공: 새벽예배 대표기도문 11월 마지막주_대표 기도문 모음 나눔터.pdf
업로드 중: 새벽예배 대표기도문 8월 넷째주_대표 기도문 모음 나눔터.pdf
✓ 성공: 새벽예배 대표기도문 8월 넷째주_대표 기도문 모음 나눔터.pdf
업로드 중: 새벽예배 대표기도문 11월 첫째주_대표 기도문 모음 나눔터.p